# Limpieza de datos

Hemos visto cómo trabajar con datos nulos eliminándolos o imputándoles la media o mediana de la variable correspondiente. Hay ocasiones en las que estos procedimientos no son los más adecuados. En esta práctica, vamos a ver otra forma de operar con datos nulos con un ejemplo sobre cáncer de mama.

Comenzamos cargando las librerías necesarias.

In [ ]:
# Si alguna no está instalada en su equipo use este código
# install.packages("ggplot2")
library(readr)
library(dplyr)
library(knitr)
library(cowplot)
library(ggplot2)

# 1. Cáncer de mama
Cuando un patólogo examina una muestra de tejido en el diagnóstico del cáncer de mama se tienen en cuenta distintas características para evaluar la agresividad del tumor. Los datos con los que vamos a trabajar proceden de [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(original)). Tenemos muestras de 699 pacientes con las siguentes variables:

- `SampleId`: Código que identifica cada muestra
- `Clump_thickness`: Espesor de la masa. Se mide como un entero entre 1 – 10.
- `Cell_Size`: Uniformidad del tamaño de la célula. Se mide como un entero entre 1 – 10.
- `Cell_Shape`: Uniformidad de la forma de la célula. Se mide como un entero entre 1 – 10.
- `Adhesion`: Adhesión marginal. Se mide como un entero entre 1 – 10.
- `Epi.Cell_Size`: (Single Epithelial Cell Size) Tamaño individual de la célula, entero entre 1 – 10.
- `Bare_Nuclei`: Núcleo, entero entre 1 – 10.
- `Bland_Chromatin`: Cromatina Blanda. Se mide como un entero entre 1 – 10.
- `Normal_Nucleoli`: Nucleoli normal. Se mide como un entero entre 1 – 10.
- `Mitoses`: Mitosis. Se mide como un entero entre 1 – 10.
- `Class`: Malignidad 2: benigno, 4: maligno

El objetivo final sería predecir si un tumor es benigno o maligno, determinado por el atributo `Class`.

In [ ]:
# Cargamos datos directamente de la web
url <- "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data"
mydata <- read.csv(file = url, header = FALSE,
                 col.names = c("ID","Clump_thickness", "Cell_Size", "Cell_Shape", "Adhesion", "Epi.Cell_Size", "Bare_Nuclei", "Bland_Chromatin", "Normal_Nucleoli","Mitoses", "Class"))

str(mydata)

# 1.1 Limpieza de datos
Vamos a bucar valores valores nulos NA. Comenzamos con una visualización de datos nulos con la función vis_miss de la librería visdat.

In [ ]:
library(visdat)
vis_miss(mydata)

A la vista del resultado, parece que no hay datos nulos NA. Veamos un resumen numérico de los datos.

In [ ]:
# Estructura y resumen numérico de los datos
str(mydata)
summary(mydata) 

Observamos que para la variable `Bare_Nuclei`hay 56 valores marcados como Other. Veamos esa variable con más detalle.

In [ ]:
# Resumen de la variable Bare_Nuclei
summary(mydata$Bare_Nuclei)

Vemos que hay valores del 1 al 10 y 16 datos marcados como ?. En este caso, no son valores NA pero esos 16 datos son nulos. Veamos cómo proceder con ellos. Discriminamos los valores de la variable `Bare_Nuclei` por el diagnóstico (`Class`: 2: benigno, 4: maligno).

In [ ]:
# Valores de la variable Bare_Nuclei por Class
p <- ggplot(mydata, aes(x = Bare_Nuclei, fill = Class)) + 
  geom_bar() + facet_grid(Class~.) +
  theme(axis.text.x = element_text(angle = 90, hjust = 1))
p

Vemos que los valores nulos ? no están distribuidos de la misma forma para benigno y maligno. Hay 2 valores ? para maligno y el resto (14) son para un diagnóstico benigno. De este modo, si queremos conservar esos registros, imputar la media o la mediana no parece muy conveniente. Además, vemos que el valor predominante para la variable `Bare_Nuclei`en benigno es 1 y la predominante en maligno es 10.

Veamos los datos con más detalle.

In [ ]:
# Extraemos los registros donde Bare_Nuclei = ?
filter(mydata, Bare_Nuclei  == "?")

Vemos que hay 16 valores ? de los que
- 14 corresponden a Class = 2 (benigno)
- 2 corresponden a Class = 4 (maligno)
Como vimos en la gráfica anterior, el valor predominante para la variable `Bare_Nuclei`en benigno es 1 y la predominante en maligno es 10. Procedemos a asignar esos valores (1 para benigno y 10 para maligno) a los valores ? de `Bare_Nuclei`.

Por otro lado, las 9 variables descriptivas están medidas en una escala ordinal del 1 al 10. La variable `Class` debemos considerarla en una escala nominal con valores 1 para maligno y 0 para benigno.

In [ ]:
# Cambiamos todos los valores ? por 1 en Bare_Nuclei
mydata$Bare_Nuclei <- factor(mydata$Bare_Nuclei,levels = c("?",1,2,3,4,5,6,7,8,9,10),
                             labels = c(1,1,2,3,4,5,6,7,8,9,10))
# Consideramos la variable Bare_Nuclei como ordinal
mydata$Bare_Nuclei <- as.integer(paste(mydata$Bare_Nuclei))

# Cambiamos los valores de la variable Class (1 = maligno, 0 = benigno)
mydata$Class <- factor(mydata$Class,levels = c(2,4),labels = c(0,1))
str(mydata)
summary(mydata)

En este momento, todos los valores ? de `Bare_Nuclei`son 1, valor predominante para un disgnóstico de benigno. Sin embargo, hemos visto 2 pacientes (con ID 1057013 y 563649) con diagnóstico maligno. A estos 2 pacientes, los valores ? de `Bare_Nuclei` deberían ser un 10, valor predominante para maligno. 

In [ ]:
# Imputamos un 10 a Bare_Nuclei en 2 pacientes
mydata$ Bare_Nuclei[mydata$ID == "1057013"] <- 10
mydata$ Bare_Nuclei[mydata$ID == "563649"] <- 10

In [ ]:
# Comprobamos
filter(mydata, ID=="1057013")
filter(mydata, ID=="563649")